<a href="https://colab.research.google.com/github/Tanu-N-Prabhu/Python/blob/master/Machine%20Learning%20Interview%20Prep%20Questions/Unsupervised%20Learning%20Algorithms/Anomaly%20Detection/One-Class%20SVM/one_class_svm_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What Is One-Class SVM?
**One-Class SVM** is an unsupervised **anomaly detection algorithm** that learns the boundary of normal data and identifies whether new observations fall **inside (normal)** or **outside (anomaly)** this boundary.

It’s based on Support Vector Machines and works well when you only have normal data during training.

## Goal

Find a function `f(x)` such that:

* `f(x) ≥ 0` for most normal data points
* `f(x) < 0` for outliers (anomalies)

We will use a **Gaussian (RBF) kernel** and solve a simplified dual form of the One-Class SVM problem.


## Assumptions for Simplicity
* Only radial basis function (RBF) kernel is used.
* No kernel matrix optimization (done via loops).
* Dual form is simplified and optimized with gradient descent.
* Output is purely educational (not production-speed).


## Step-by-Step Implementation




In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Generate Normal Training Data (Only Normal Data)

In [2]:
np.random.seed(42)

# Generate normal data (centered around origin)
X_train = np.random.normal(0, 1, size=(100, 2))

# Test data: normal + outliers
X_test_normal = np.random.normal(0, 1, size=(10, 2))
X_test_outliers = np.random.uniform(low=4, high=6, size=(5, 2))
X_test = np.vstack((X_test_normal, X_test_outliers))

## 2. Define RBF (Gaussian) Kernel

In [3]:
def rbf_kernel(x1, x2, gamma=0.5):
    """Compute RBF kernel between two vectors."""
    distance_squared = np.linalg.norm(x1 - x2) ** 2
    return np.exp(-gamma * distance_squared)

## 3. Compute Full Kernel Matrix

In [4]:
def compute_kernel_matrix(X, gamma=0.5):
    n_samples = X.shape[0]
    K = np.zeros((n_samples, n_samples))

    for i in range(n_samples):
        for j in range(n_samples):
            K[i, j] = rbf_kernel(X[i], X[j], gamma)
    return K

## Train One-Class SVM (Dual Problem)
We optimize the dual objective:

We optimize the **dual form** of the One-Class SVM:

$$[
\max_{\alpha} \sum_{i=1}^{n} \alpha_i K(x_i, x_i) - \sum_{i=1}^{n} \sum_{j=1}^{n} \alpha_i \alpha_j K(x_i, x_j)
]$$

Subject to the constraints:

$$[
0 \leq \alpha_i \leq \frac{1}{\nu n}, \quad \sum_{i=1}^{n} \alpha_i = 1
]$$

Where:
- $$( \alpha_i )$$ are the Lagrange multipliers (weights for each support vector)
- $$( K(x_i, x_j) )$$ is the **RBF kernel** (or any other kernel)
- $$( \nu \in (0, 1] )$$ controls the fraction of outliers
- $$( n )$$ is the number of training samples


In this notebook, we use **gradient descent** to optimize the dual objective:

- Initialize $$( \alpha_i = \frac{1}{n} )$$
- Update using the gradient:
  $$[
  \alpha_i \leftarrow \alpha_i - \eta \cdot \left( \frac{\partial L}{\partial \alpha_i} \right)
  ]$$
- Project $$( \alpha )$$ back to the feasible region:
  - Clip to range $$( [0, \frac{1}{\nu n}] )$$
  - Normalize to ensure $$( \sum \alpha_i = 1 )$$ using **simplex projection**

This gives us a working set of $$( \alpha )$$ weights that can be used for scoring test points.



In [5]:
def train_one_class_svm(X, nu=0.1, gamma=0.5, lr=0.01, n_iters=100):
    n_samples = X.shape[0]
    K = compute_kernel_matrix(X, gamma)

    alpha = np.full(n_samples, 1 / n_samples)

    # Gradient descent loop
    for _ in range(n_iters):
        gradient = K @ alpha
        gradient = gradient * 2 - np.diag(K)

        # Update alpha
        alpha -= lr * gradient

        # Clip values to constraint [0, 1/(nu * n)]
        alpha = np.clip(alpha, 0, 1 / (nu * n_samples))

        # Project onto simplex so sum(alpha) = 1
        alpha = project_to_simplex(alpha)

    return alpha, K

## 5. Project to Simplex (Ensures sum of α = 1)


In [6]:
def project_to_simplex(v):
    """Project vector v to the probability simplex (sum = 1, v_i >= 0)."""
    n = len(v)
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, n + 1) > (cssv - 1))[0][-1]
    theta = (cssv[rho] - 1) / (rho + 1)
    return np.maximum(v - theta, 0)

## 6. Compute Decision Function



In [7]:
def decision_function(x, X_train, alpha, gamma):
    result = 0
    for i in range(len(X_train)):
        result += alpha[i] * rbf_kernel(x, X_train[i], gamma)
    return result

## 7. Predict Labels for New Data


In [8]:
def predict(X_test, X_train, alpha, gamma, threshold):
    predictions = []
    scores = []

    for x in X_test:
        score = decision_function(x, X_train, alpha, gamma)
        label = 1 if score >= threshold else -1  # 1 = normal, -1 = anomaly
        predictions.append(label)
        scores.append(score)

    return predictions, scores

## 8. Run Everything

In [9]:
# Train model
gamma = 0.5
nu = 0.1
alpha, K_train = train_one_class_svm(X_train, nu=nu, gamma=gamma)

# Use average of decision scores on training set as threshold
train_scores = [decision_function(x, X_train, alpha, gamma) for x in X_train]
threshold = np.percentile(train_scores, 100 * nu)

# Predict
predictions, scores = predict(X_test, X_train, alpha, gamma, threshold)

# Display
for i, (x, pred, score) in enumerate(zip(X_test, predictions, scores)):
    status = "Normal ✅" if pred == 1 else "Anomaly 🚨"
    print(f"Test Point {i}: {x} | Score: {score:.4f} | {status}")

Test Point 0: [0.35778736 0.56078453] | Score: 0.2054 | Normal ✅
Test Point 1: [1.08305124 1.05380205] | Score: 0.2053 | Normal ✅
Test Point 2: [-1.37766937 -0.93782504] | Score: 0.2043 | Normal ✅
Test Point 3: [0.51503527 0.51378595] | Score: 0.2071 | Normal ✅
Test Point 4: [0.51504769 3.85273149] | Score: 0.0619 | Anomaly 🚨
Test Point 5: [0.57089051 1.13556564] | Score: 0.2124 | Normal ✅
Test Point 6: [0.95400176 0.65139125] | Score: 0.2130 | Normal ✅
Test Point 7: [-0.31526924  0.75896922] | Score: 0.2054 | Normal ✅
Test Point 8: [-0.77282521 -0.23681861] | Score: 0.2119 | Normal ✅
Test Point 9: [-0.48536355  0.08187414] | Score: 0.2037 | Normal ✅
Test Point 10: [4.93119604 5.08528927] | Score: 0.0000 | Anomaly 🚨
Test Point 11: [4.5730825  5.18166652] | Score: 0.0000 | Anomaly 🚨
Test Point 12: [4.0610005  4.07469638] | Score: 0.0001 | Anomaly 🚨
Test Point 13: [5.64520112 4.72038128] | Score: 0.0000 | Anomaly 🚨
Test Point 14: [4.25412103 5.04448652] | Score: 0.0000 | Anomaly 🚨


## Summary

| Step | Action                                         |
|:---- |:---------------------------------------------- |
| 1    | Generate normal and test data                  |
| 2    | Define RBF kernel                              |
| 3    | Compute kernel matrix                          |
| 4    | Solve dual optimization using gradient descent |
| 5    | Project weights onto simplex (sum = 1)         |
| 6    | Compute anomaly scores using decision function |
| 7    | Label points with threshold                    |
